### Установка пакетов

In [1]:
!pip install pyspark

### Импорт библиотек

In [2]:
import json
import os
from pyspark.ml.regression import LinearRegression
from pyspark.sql import SparkSession
from sklearn.datasets import make_classification

### Создание каталога data для хранения данных

In [3]:
if not os.path.exists('data'):
  os.makedirs('data')

### Создание экземпляра SparkSession

In [4]:
spark = SparkSession.builder.appName('Homework_3_lab3').getOrCreate()

## Регрессия (LinearRegression)

### Задание 1. Сгенерировать подходящие исходные данные для проведения обучения

#### 1.1. Определение функции генерации данных

In [5]:
def to_file(path, X, y):
  lines = []

  for i in range(len(y)):
    features = []

    for j in range(len(X[0])):
      features.append(f"{j+1}:{X[i][j]}")

    lines.append(f"{y[i]} {' '.join(features)}")

  with open(path, "w") as file:
    file.write('\n'.join(lines))

#### 1.2. Генерация данных

In [6]:
X, y = make_classification(
  n_samples=1000,
  n_features=15,
  n_informative=12,
  n_redundant=3,
  n_clusters_per_class=2,
  n_classes=5
)

to_file('data/regrassion_data', X, y)

#### 1.3. Чтение данных из файла

In [7]:
df = spark.read.format('libsvm').load('data/regrassion_data')
df.head()

Row(label=0.0, features=SparseVector(15, {0: -5.1786, 1: -0.6475, 2: 1.7988, 3: 4.3444, 4: -3.0755, 5: 3.096, 6: -1.0053, 7: 1.4436, 8: -3.3272, 9: 3.5888, 10: 2.291, 11: -0.8982, 12: -4.2056, 13: 1.9379, 14: -7.7826}))

### Заданние 2. Провести кластеризацию с помощью pyspark.ml / pyspark.mllib

In [8]:
df_train, df_test = df.randomSplit([0.8, 0.2])

linear = LinearRegression()
model = linear.fit(df_train)

In [9]:
predictions = model.transform(df_test)
predictions.head()

Row(label=0.0, features=SparseVector(15, {0: -9.1907, 1: 2.5275, 2: -1.5476, 3: 6.2502, 4: 0.3802, 5: 0.8253, 6: -1.4836, 7: -3.8603, 8: -0.5895, 9: 0.8325, 10: 0.7595, 11: 1.3631, 12: 1.977, 13: 1.9326, 14: -1.6815}), prediction=2.2970614446496254)

In [10]:
model.coefficients

DenseVector([-0.0013, 0.0486, -0.0477, -0.0076, -0.032, -0.0418, -0.1531, -0.0974, 0.1118, -0.0263, 0.0013, -0.0902, -0.0639, -0.0366, 0.0146])

In [11]:
model.intercept

2.0107825841826372

### Заданние 3. Провести расчет метрик

In [12]:
model.summary.r2

0.1827139155140628

In [13]:
model.summary.rootMeanSquaredError

1.2728786683653008

### Задание 4. Сохранить полученные значения метрик в файл.

In [14]:
metrics_list = [ model.summary.r2, model.summary.rootMeanSquaredError ]

with open('data/regrassion_data_r_3', 'w') as file:
  file.write(str(metrics_list))